<a href="https://colab.research.google.com/github/jayw20230711/COLAB/blob/main/kg-rag-Ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os

# --- Configuration ---
remote_pdf_url = "https://arxiv.org/pdf/1709.00666.pdf"
pdf_filename = "ch02-downloaded.pdf"

print(f"Attempting to download PDF from: {remote_pdf_url}")

try:
    # Use a timeout to prevent the script from hanging indefinitely
    response = requests.get(remote_pdf_url, timeout=15)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Write the content of the response to a file in binary mode ('wb')
        with open(pdf_filename, "wb") as pdf_file:
            pdf_file.write(response.content)

        # Confirmation message
        print(f"✅ Success! PDF saved as: {os.path.abspath(pdf_filename)}")
    else:
        # Handle non-200 status codes (e.g., 404 Not Found, 500 Server Error)
        print(f"❌ Failed to download the PDF. HTTP Status Code: {response.status_code}")

except requests.exceptions.RequestException as e:
    # Handle connection errors, DNS errors, timeouts, etc.
    print(f"❌ An error occurred during the request: {e}")

except IOError as e:
    # Handle file writing errors
    print(f"❌ An error occurred while writing the file: {e}")

Attempting to download PDF from: https://arxiv.org/pdf/1709.00666.pdf
✅ Success! PDF saved as: /content/ch02-downloaded.pdf


In [ ]:
!pip install pdfplumber
import pdfplumber
import os

# --- Configuration (Must match the name used in the download script) ---
pdf_filename = "ch02-downloaded.pdf"
text = ""

# Check if the PDF file exists before trying to open it
if not os.path.exists(pdf_filename):
    print(f"❌ Error: File not found. Please ensure '{pdf_filename}' was downloaded successfully in the previous step.")
else:
    try:
        print(f"Reading text from: {pdf_filename}...")

        with pdfplumber.open(pdf_filename) as pdf:
            for i, page in enumerate(pdf.pages):
                # Using += might be memory-intensive for huge files,
                # but is fine for this example.
                extracted_page_text = page.extract_text()
                if extracted_page_text:
                    text += extracted_page_text + "\n"

        # Check if any text was extracted
        if text:
            print(f"✅ Success! Extracted {len(text)} characters.")
            print("\n--- First 20 characters of the extracted text ---")
            print(text[0:20])
        else:
            print("⚠️ Warning: Successfully opened the PDF, but no text could be extracted.")

    except Exception as e:
        print(f"❌ An unexpected error occurred while processing the PDF: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.7 MB/s eta 0:00:00
Reading text from: ch02-downloaded.pdf...
✅ Success! Extracted 44774 characters.

--- First 20 characters of the extracted text ---
Einstein’s Patents a


In [ ]:
# If you run into ModuleNotFoundError, uncomment the line below and run it first
!pip install google-genai neo4j python-dotenv

import os
from google import genai
from neo4j import GraphDatabase
from google.colab import userdata

# --- 1. Set up the Gemini (GenAI) client ---

# Retrieve API key from Colab's secrets manager
gemini_api_key = userdata.get("GEMINI_API_KEY")

if not gemini_api_key:
    # Print a clear error if the key is missing
    print("❌ ERROR: GEMINI_API_KEY not found in Colab secrets.")
    print("Please ensure you have saved 'GEMINI_API_KEY' in Colab's secrets manager (🔑 icon on the left panel).")
    genai_client = None
else:
    print("✅ GEMINI_API_KEY loaded successfully from Colab secrets.")
    # Initialize the client
    genai_client = genai.Client(api_key=gemini_api_key)
    print("✅ Gemini Client initialized.")

# NOTE: You would typically add the Neo4j setup here as well,
# but for now, we focus on fixing the Gemini client part.

# Example of how you would initialize Neo4j (if needed later)
# NEO4J_URI = os.getenv("NEO4J_URI")
# NEO4J_USER = os.getenv("NEO4J_USER")
# NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
# neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

✅ GEMINI_API_KEY loaded successfully from Colab secrets.
✅ Gemini Client initialized.


In [1]:
def chunk_text(text, chunk_size, chunk_overlap):
    chunks = []
    # A simple way to split text, can be improved with sentence tokenizers for better results
    # or using a more sophisticated text splitter like those from LangChain.
    text_splits = text.split('\n')
    current_chunk = []
    current_length = 0

    for split in text_splits:
        # Check if adding the next split would exceed the chunk_size
        # +1 for the newline character if re-joined for length calculation
        if current_length + len(split) + 1 > chunk_size and current_chunk:
            chunks.append("\n".join(current_chunk))
            # Create overlap: start the new chunk with the last 'chunk_overlap' lines
            if chunk_overlap > 0:
                overlap_lines = current_chunk[-chunk_overlap:]
                current_chunk = overlap_lines
                current_length = sum(len(line) + 1 for line in overlap_lines) # +1 for newline
            else:
                current_chunk = []
                current_length = 0

        current_chunk.append(split)
        current_length += len(split) + 1 # Add length of current split + newline

    # Add any remaining text as the last chunk
    if current_chunk:
        chunks.append("\n".join(current_chunk))

    return chunks

chunks = chunk_text(text, 500, 40)
print(len(chunks))
print(chunks[0])

ModuleNotFoundError: No module named 'utils'